# Create k-mer matrix

This notebook creates the k-mer count matrix for 
N. gonorrhoeae
Azithromycin

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import pylab as plt
import glob
import os
from scipy.sparse import lil_matrix

### Load k-mers from DSK output

Point this to the folder containing all the .txt files

In [3]:
data_dir = '/home/joewandy/Downloads/txt_archive/kmers/'

In [4]:
filenames = sorted(glob.glob(os.path.join(data_dir, '*.txt')))

In [9]:
dfs = []
for i in range(len(filenames)):
    filename = filenames[i]
    
    # load the output from dsk2ascii
    print('%d/%d %s' % (i+1, len(filenames), filename))
    df = pd.read_csv(filename, header=None, sep=' ')
    df = df.rename(columns={0: 'kmer', 1: 'count'})
    
    # get strain information from filename
    basename = os.path.basename(filename)
    strain = '.'.join(basename.split('.')[0:-1])
    df['strain'] = strain
    
    # reorder columns
    cols = ['strain', 'kmer', 'count']
    df = df[cols]
    
    dfs.append(df)

1/392 /home/joewandy/Downloads/txt_archive/kmers/485.291.txt
2/392 /home/joewandy/Downloads/txt_archive/kmers/485.292.txt
3/392 /home/joewandy/Downloads/txt_archive/kmers/485.294.txt
4/392 /home/joewandy/Downloads/txt_archive/kmers/485.295.txt
5/392 /home/joewandy/Downloads/txt_archive/kmers/485.298.txt
6/392 /home/joewandy/Downloads/txt_archive/kmers/485.299.txt
7/392 /home/joewandy/Downloads/txt_archive/kmers/485.301.txt
8/392 /home/joewandy/Downloads/txt_archive/kmers/485.305.txt
9/392 /home/joewandy/Downloads/txt_archive/kmers/485.307.txt
10/392 /home/joewandy/Downloads/txt_archive/kmers/485.311.txt
11/392 /home/joewandy/Downloads/txt_archive/kmers/485.312.txt
12/392 /home/joewandy/Downloads/txt_archive/kmers/485.313.txt
13/392 /home/joewandy/Downloads/txt_archive/kmers/485.314.txt
14/392 /home/joewandy/Downloads/txt_archive/kmers/485.316.txt
15/392 /home/joewandy/Downloads/txt_archive/kmers/485.317.txt
16/392 /home/joewandy/Downloads/txt_archive/kmers/485.318.txt
17/392 /home/joew

144/392 /home/joewandy/Downloads/txt_archive/kmers/485.532.txt
145/392 /home/joewandy/Downloads/txt_archive/kmers/485.533.txt
146/392 /home/joewandy/Downloads/txt_archive/kmers/485.534.txt
147/392 /home/joewandy/Downloads/txt_archive/kmers/485.535.txt
148/392 /home/joewandy/Downloads/txt_archive/kmers/485.536.txt
149/392 /home/joewandy/Downloads/txt_archive/kmers/485.537.txt
150/392 /home/joewandy/Downloads/txt_archive/kmers/485.538.txt
151/392 /home/joewandy/Downloads/txt_archive/kmers/485.539.txt
152/392 /home/joewandy/Downloads/txt_archive/kmers/485.540.txt
153/392 /home/joewandy/Downloads/txt_archive/kmers/485.541.txt
154/392 /home/joewandy/Downloads/txt_archive/kmers/485.542.txt
155/392 /home/joewandy/Downloads/txt_archive/kmers/485.543.txt
156/392 /home/joewandy/Downloads/txt_archive/kmers/485.544.txt
157/392 /home/joewandy/Downloads/txt_archive/kmers/485.545.txt
158/392 /home/joewandy/Downloads/txt_archive/kmers/485.546.txt
159/392 /home/joewandy/Downloads/txt_archive/kmers/485.

288/392 /home/joewandy/Downloads/txt_archive/kmers/485.769.txt
289/392 /home/joewandy/Downloads/txt_archive/kmers/485.770.txt
290/392 /home/joewandy/Downloads/txt_archive/kmers/485.771.txt
291/392 /home/joewandy/Downloads/txt_archive/kmers/485.772.txt
292/392 /home/joewandy/Downloads/txt_archive/kmers/485.773.txt
293/392 /home/joewandy/Downloads/txt_archive/kmers/485.774.txt
294/392 /home/joewandy/Downloads/txt_archive/kmers/485.775.txt
295/392 /home/joewandy/Downloads/txt_archive/kmers/485.776.txt
296/392 /home/joewandy/Downloads/txt_archive/kmers/485.777.txt
297/392 /home/joewandy/Downloads/txt_archive/kmers/485.778.txt
298/392 /home/joewandy/Downloads/txt_archive/kmers/485.779.txt
299/392 /home/joewandy/Downloads/txt_archive/kmers/485.780.txt
300/392 /home/joewandy/Downloads/txt_archive/kmers/485.781.txt
301/392 /home/joewandy/Downloads/txt_archive/kmers/485.782.txt
302/392 /home/joewandy/Downloads/txt_archive/kmers/485.783.txt
303/392 /home/joewandy/Downloads/txt_archive/kmers/485.

In [10]:
df = pd.concat(dfs)

In [11]:
df

,strain,kmer,count
0,485.291,AAAAAATGCCGTCTGAAGGCTTCAGACGGCA,2
1,485.291,AAAAATGCCGTCTGAAAGCCCTTCAGACGGC,2
2,485.291,AAAAATGCCGTCTGAAAGCCTTTCAGACGGC,4
3,485.291,AAAAATGCCGTCTGAAGACCTTTCAGACGGC,2
4,485.291,AAAAATGCCGTCTGAAGGCTTCAGACGGCAT,4
5,485.291,AAAATGCCGTCTGAAAGCCCTTCAGACGGCA,2
6,485.291,AAAATGCCGTCTGAAAGCCTTTCAGACGGCA,6
7,485.291,AAAATGCCGTCTGAAAGGCTTTCAGACGGCA,5
8,485.291,AAAATGCCGTCTGAAAGGTCTTCAGACGGCA,2
9,485.291,AAAATGCCGTCTGAAAGGGCTTCAGACGGCA,2


### Construct a feature matrix

Get unique strains and kmers and index them

In [ ]:
strains = sorted(df['strain'].unique())
kmers = sorted(df['kmer'].unique())
n_strains = len(strains)
n_kmers = len(kmers)

strain_idx = { k: v for v, k in list(enumerate(strains)) }
kmer_idx = { k: v for v, k in list(enumerate(kmers)) }

Populate sparse matrix. Very slow!!

In [ ]:
feature = lil_matrix((n_strains, n_kmers))

total = df.shape[0]
counter = 0
for idx, row in df.iterrows():
    counter += 1
    if counter % 100000 == 0:
        progress = counter / total * 100
        print('%.1f%%' % progress)    
    
    strain = row['strain']
    kmer = row['kmer']
    count = row['count']
            
    i = strain_idx[strain]
    j = kmer_idx[kmer]
    feature[i, j] = count

### Same as above but much faster using pandas

In [12]:
feature_df = df.pivot(index='strain', columns='kmer', values='count')
feature_df = feature_df.fillna(0) # replace NaN with 0

In [18]:
feature_df.shape

(392, 269530)

In [20]:
feature_df.head(10)

kmer,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAG,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAGA,AAAAAAAAAAAAAAAAAAAAAAAAAAAAGAA,AAAAAAAAAAAAAAAAAAAAAAAAAAAGAAT,AAAAAAAAAAAAAAAAAAAAAAAAAAGAATT,AAAAAAAAAAAAAAAAAAAAAAAAAGAATTG,AAAAAAAAAAAAAAAAAAAAAAAAGAATTGC,AAAAAAAAAAAAAAAAAAAAAAAGAATTGCT,AAAAAAAAAAAAAAAAAAAAAAGAATTGCTT,...,TTTTTTTTTGACGGCAGGGGGCAACGTTTCA,TTTTTTTTTGCCCGTTTTTGAGGATTCGCAA,TTTTTTTTTGCCCGTTTTTGAGGATTCGCAC,TTTTTTTTTGGGCGTATCTCAGTCCGGAATC,TTTTTTTTTTCAGACGACCTGAAACAACAGC,TTTTTTTTTTGCCCGTTTTTGAGGATTCGCA,TTTTTTTTTTTTCAAGCAGAAGACGGCATAC,TTTTTTTTTTTTTTTCAAGCAGAAGACGGCA,TTTTTTTTTTTTTTTTCAAGCAGAAGACGGC,TTTTTTTTTTTTTTTTTTTCAAGCAGAAGAC
strain,,,,,,,,,,,,,,,,,,,,,
485.291,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.292,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.295,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.298,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.299,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.301,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.305,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485.307,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Load target prediction

In [23]:
predicted_df = pd.read_csv('/home/joewandy/Downloads/metadata.tsv', sep='\t', header=None)
predicted_df = predicted_df.rename(columns={0: 'strain', 1: 'resistance'})
predicted_df.set_index('strain')

,resistance
strain,
485.874,sensitive
485.757,sensitive
485.409,sensitive
485.756,sensitive
485.755,sensitive
485.754,sensitive
485.753,sensitive
485.751,sensitive
485.415,sensitive
